# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv("output_data/cities_weather.csv")
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saskylakh,71.9167,114.0833,34.38,97,100,19.57,RU,1631812804
1,Hithadhoo,-0.6000,73.0833,83.91,70,100,9.78,MV,1631812805
2,Moscow,55.7522,37.6156,46.83,67,100,6.93,RU,1631812729
3,Rikitea,-23.1203,-134.9692,73.17,77,7,14.70,PF,1631812807
4,Bluff,-46.6000,168.3333,36.81,73,70,11.95,NZ,1631812807
...,...,...,...,...,...,...,...,...,...
561,Pandan,14.0453,124.1694,78.01,87,100,1.39,PH,1631813333
562,Tabas,33.5959,56.9244,88.90,8,0,7.63,IR,1631813334
563,La Paz,-16.5000,-68.1500,58.66,44,82,11.50,BO,1631813335
564,Kupang,-10.1667,123.5833,78.44,81,74,9.31,ID,1631813336


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1)
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = city_df.loc[(city_df["Wind Speed"] <= 10) & (city_df["Cloudiness"] == 0) & \
                                   (city_df["Max Temp"] >= 70) & (city_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,Gaza,31.5000,34.4667,79.92,60,0,6.71,PS,1631812811
86,Chardara,41.2547,67.9692,74.19,28,0,8.66,KZ,1631812878
180,Neuquén,-38.9516,-68.0591,71.49,24,0,9.22,AR,1631812967
297,Goderich,43.7501,-81.7165,74.91,49,0,6.91,CA,1631813076
340,Belle Fourche,44.6714,-103.8522,71.31,35,0,6.82,US,1631813122
354,Beloha,-25.1667,45.0500,70.23,70,0,7.00,MG,1631813134
384,Yangi Marg`ilon,40.4272,71.7189,72.32,33,0,4.61,UZ,1631813161
493,Ovalle,-30.5983,-71.2003,75.13,28,0,6.04,CL,1631813266
505,Susanville,40.4163,-120.6530,70.09,40,0,1.01,US,1631813280
525,Vallenar,-28.5708,-70.7581,78.31,21,0,7.56,CL,1631813299


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,Gaza,PS,31.5000,34.4667,
86,Chardara,KZ,41.2547,67.9692,
180,Neuquén,AR,-38.9516,-68.0591,
297,Goderich,CA,43.7501,-81.7165,
340,Belle Fourche,US,44.6714,-103.8522,
354,Beloha,MG,-25.1667,45.0500,
384,Yangi Marg`ilon,UZ,40.4272,71.7189,
493,Ovalle,CL,-30.5983,-71.2003,
505,Susanville,US,40.4163,-120.6530,
525,Vallenar,CL,-28.5708,-70.7581,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
 
    results = response['results']

    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")    

    time.sleep(1)

print("-------End of Search-------")


Retrieving Results for Index 9: Gaza.
Missing field/result... skipping.
------------
Retrieving Results for Index 86: Chardara.
Missing field/result... skipping.
------------
Retrieving Results for Index 180: Neuquén.
Missing field/result... skipping.
------------
Retrieving Results for Index 297: Goderich.
Missing field/result... skipping.
------------
Retrieving Results for Index 340: Belle Fourche.
Missing field/result... skipping.
------------
Retrieving Results for Index 354: Beloha.
Missing field/result... skipping.
------------
Retrieving Results for Index 384: Yangi Marg`ilon.
Missing field/result... skipping.
------------
Retrieving Results for Index 493: Ovalle.
Missing field/result... skipping.
------------
Retrieving Results for Index 505: Susanville.
Missing field/result... skipping.
------------
Retrieving Results for Index 525: Vallenar.
Missing field/result... skipping.
------------
Retrieving Results for Index 553: Mercedes.
Missing field/result... skipping.
----------

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))